In [ ]:
%run ../supportvectors-common.ipynb

In [2]:
import torch
from torchvision.transforms import v2
from svlearn.config.configuration import ConfigurationMixin
                                                        
from svlearn.auto_encoders.resnet_auto_encoder import ResNetAutoEncoder

In [3]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
config = ConfigurationMixin().load_config()
results_dir = config['tree-classification']['results']

In [5]:
test_transform = v2.Compose([
    v2.ToImage(), 
    v2.Resize(size=(224 , 224)),  # resize all images to a standard size suitable for the cnn model
    v2.ToDtype(torch.float32, scale=True), # ensure te tensor is of float datatype
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # normalize tensor 
])

In [13]:
# Load the images with some artificially induced noise
import os
from PIL import Image
image_path = '/home/chandar/images'
images = []
for filename in os.listdir(image_path):
    file_path = os.path.join(image_path, filename)
    img = Image.open(file_path).convert('RGB')  # Ensure 3-channel RGB format
    # Apply transformations
    img_transformed = test_transform(img)
    noisy_image = img_transformed + 0.0 * torch.randn_like(img_transformed)
    images.append(noisy_image)


In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = ResNetAutoEncoder().to(device)

In [8]:
checkpoint = torch.load(f"{results_dir}/trees_resnet50_denoising_autoencoder.pt")

In [ ]:
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

In [ ]:
import matplotlib.pyplot as plt
from svlearn.auto_encoders.auto_encoder_util import convert
with torch.no_grad():
    num_images = len(images)
    images = [img.to(device) for img in images]
    images = torch.stack(images)
    reconstructed, _ = model(images)
    
    # Display original and reconstructed images
    _, axs = plt.subplots(2, num_images, figsize=(10, 4))
    for i in range(num_images):
        axs[0, i].imshow(convert(images[i].cpu()))
        axs[1, i].imshow(convert(reconstructed[i].cpu()))
    plt.show()